##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [ ]:
!pip install -U -q "google-generativeai>=0.8.2"

In [ ]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-pro' # @param {isTemplate: true}
contents_b64 = 'W10=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Upload the video and print a confirmation.
video_file_name = "/content/drive/MyDrive/经济学/MIT微观_x264.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")
import time

# Check whether the file is ready to be used.
while video_file.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/8f6j3xwzea12
...........

In [ ]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

# Create the prompt.
prompt = """角色设定: 你是一个学生，正在学习刑法课，想要做到能够清晰、准确地理解法律文本，并能结合实际案例进行分析。
任务描述: 请观看我下面给出的刑法总论的关于贿赂犯罪的课程视频。在观看完视频后，请回答下列问题。确保你的回答详细、准确，并结合视频中的相关信息来支持你的答案。
"""
contents = [prompt]

print("Making LLM inference request...")
response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)
print("Successfully generating reponse:")
display(Markdown(response.text))

Making LLM inference request...
Successfully generating reponse:


请您提供刑法总论关于贿赂犯罪的课程视频。我看完后会尽力按照您的要求，以学生的身份详细、准确地回答您的问题，并结合视频内容进行分析。


## Call `generate_content`

In [ ]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

# Create the prompt.
prompt = """
Role: You are a very useful video assistant who can watch video content and answer questions related to it.
Task: Please watch the following news report video and answer the questions listed below based solely on the content of the video. Provide clear and accurate answers. The format should list the question first, followed by the answer.
Question list：
1.	How many active fires are currently burning in Los Angeles?
2.	What are the two main environmental factors that created conditions for the fires to spread?
3.	What type of winds are particularly concerning in the area?
4.	What measures have been taken to protect abandoned properties?
5.	How have local communities responded to the emergency services being stretched thin?
6.	What resources are the local volunteers using to help fight the fires?
7.	Why did local volunteers feel their involvement was particularly valuable?
8.	What is the geographical location of Altadena relative to the danger?
9.	What historical climate milestone was reached last year according to the report?
10.	What warning is given about potential deaths?
11.	What is the mood among residents about rebuilding?
12.	What is the current state of the emergency services according to the report?
13.	How do we know that both wealthy and less affluent areas are affected by the fires, and what specific areas illustrate this contrast?
14.	What evidence does the report provide about both immediate and long-term climate concerns?
15.	What was Stephanie's attitude toward the future despite losing her home?
16.	Who is Tavia Weinman and what was notable about her situation? What specific item was she looking for in the ruins?
17.	What are the five fires currently burning, and which one is located the furthest south and what is its containment level?
18.	What firefighting methods can you see being implemented from the video?
19.	How is the reporter Emma Vardy dressed in the video?
20.	What were the scenes on Fair Oaks Avenue like before and after the fire, please describe according to the video footage.
21.	How does the video vividly illustrate the changes in average daily temperatures since 1940, and what is the trend of these changes?
22.	Who in the report says, "It's the warmest period in at least 100,000 years."

"""
contents = [video_file, prompt]

print("Making LLM inference request...")
response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)
print("Successfully generating reponse:")
display(Markdown(response.text))

Making LLM inference request...
Successfully generating reponse:


Sure! Here are the answers to your questions based on the provided video clip.

**CONCEPT PROBLEMS:**
1. The primary objective of firms is to maximize their profits, which is defined as revenue minus costs.


2. The difference between the short-run and the long-run from the perspective of production theory is that in the short-run, some inputs are fixed, while in the long-run, all inputs are variable.


3. All of the listed options are properties of isoquants.


4. A production function in which doubling all inputs causes output to more than double is known as increasing returns to scale.


5. In the short run, the marginal cost is defined as the wage rate multiplied by the number of hours needed to produce one additional unit.


6. An isocost line is defined as combinations of labor and capital that yield the same levels of cost.


7. The economically efficient point of production for firms is identified by the tangency of an isoquant with an isocost line.


8. The example of McDonald’s demonstrates the concept of labor becoming less productive, not capital. This occurs in a short-run analysis. In a long-run analysis, both capital and labor are variable, as both can be changed. With the two isoquants given, if capital becomes less productive as the business expands, the slope of the long-run expansion path will decrease as output increases, or as L increases. This means the firm will become more capital-intensive with more expansion. The long-run expansion path would be concave.


9. The cost of a law school education would be considered sunk costs. This is because these are expenses that can not be recovered once you have graduated law school.


10. Short-run choices are 1, and 3. Long-run choices are 2, 4 and 5.


11. In the long-run, all inputs are variable. Since fixed costs are defined as costs that do not vary with the level of output, there can be no fixed costs in the long run. 


12. The manager should assess whether each worker is worth less than the previous workers by using the marginal product of labor. After calculating the value of each worker and calculating the cost of an additional worker, the manager can then evaluate the options of increasing the number of workers or adding machines.


13. Adding a 19-year-old student to the class will decrease the average age from 23.7 years. This is an example of diminishing marginal product. The average value decreases as each marginal value is added.


14. Barry Bond’s career home run average increased after the 2001 season. His performance increased his home run average. This is because the marginal value (73 home runs) was higher than the average value (33 home runs). The average increases with each marginal value if the marginal value is higher than the average value.


15. If the firm is at the minimum point of its average cost curve, that means the marginal cost is equal to the average cost. The firm would not alter the size of its plant unless the prices of the inputs changed.


16. The difference between average total cost and average variable cost decreases with each unit of production, because the average fixed costs decrease with each unit of production, and therefore decreases the difference between average total cost and average variable cost.


17. This information is not covered in the lecture clip.


18. Out of the listed firms, you would expect diseconomies of scale to set in at relatively low levels of output for the restaurant, copy shop and newspaper. This is due to higher fixed costs, which need to be spread out amongst many units of production, and also due to the diminishing marginal returns of labor that result from a lack of machines to aid in production.


19. As car repair shops require more computerized testing equipment to repair newer cars, the long-run average total cost curves will likely rise, as the increased use of technology increases costs.


**NUMERICAL PROBLEMS:**
1. 
1. The marginal product of the second janitor is 4 classrooms.
2. The average product of four janitors is 4 classrooms.
3. The addition of the third janitor is associated with decreasing marginal returns because the marginal product of labor is less than the previous marginal product. 
4. The addition of the fourth janitor is associated with decreasing marginal returns because the marginal product of labor is less than the previous marginal product.
5. The addition of the seventh janitor is associated with negative marginal returns because the total product decreases as a result.
6. The image can’t be provided since the video did not show the drawing process.
7. The slope of the total product curve is the marginal product of labor. Therefore, the slope values are 3, 4, 5, 4, 1, 0, and -1.
8.
1. The marginal product of labor increases when the slope of the total product curve is positive and increasing.
2. The marginal product of labor decreases when the slope of the total product curve is positive and decreasing.
3. The marginal product of labor is negative when the slope of the total product curve is negative.


2. The total cost is $150,000.


3. The error in the director’s recommendation is that she included overhead costs. Overhead costs are a type of sunk costs, which should not be included when making decisions about production.


4. 
1. The average fixed cost of cleaning three classrooms is $33.33.
2. The average variable cost of cleaning three classrooms is $33.33.
3. The average fixed cost of cleaning seven classrooms is $14.29.
4. The average variable cost of cleaning seven classrooms is $28.57.
5. The marginal cost of cleaning the seventeenth classroom is $100.
6. The average total cost of cleaning twelve classrooms is $33.33.


5. The information provided is insufficient to sketch the curves. The variable costs of publishing magazines are labor, paper and ink, electricity and marketing. Fixed costs are printing presses and building rent.


6. The calculations are complex and can’t be provided without more detailed information.


7. 
1. The image can’t be provided, since the video did not show the drawing process.
2. The image can’t be provided.
3. The cost per knife is decreasing in the region of the long-run average cost curve that corresponds to economies of scale.
4. The cost per knife is constant in the region of the long-run average cost curve that corresponds to constant returns to scale.
5. The cost per knife is increasing in the region of the long-run average cost curve that corresponds to diseconomies of scale.


8. If the marginal product of capital is 60 and the price of capital is $6, and the marginal product of labor is 20 and the price of labor is $2.50, the firm should increase the amount of labor and decrease the amount of capital used. This is because the bang-for-the-buck of labor is 8, while the bang-for-the-buck of capital is 10. Therefore, the firm can achieve a greater marginal product per dollar using capital.


9. The price of labor must be $5 per unit.


10.
1. The marginal product of capital must be 100.
2. The firm would become more labor-intensive since capital becomes more expensive to use relative to labor. 


I hope this is helpful. Let me know if you have any other questions.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")